# Tarefa 2: Usar o SageMaker Feature Store

## Tarefa 2.1: Opções do Feature Store

Há três maneiras principais de armazenar recursos no Amazon SageMaker:
1. Usar o Amazon SageMaker Feature Store como destino do Amazon SageMaker Data Wrangler depois que as etapas de pré-processamento forem concluídas e os recursos forem adicionados.
2. Exportar um notebook do SageMaker Data Wrangler que executa definição de recurso, criação de grupo de recursos e ingestão de dados no SageMaker Feature Store.
3. Usar o SDK Python do SageMaker em um notebook personalizado que executa definição de recurso, criação de grupo de recursos e ingestão de dados no SageMaker Feature Store.

Essas três opções estão descritas nas seções a seguir.

### Usar o SageMaker Feature Store como um destino do SageMaker Data Wrangler

É possível adicionar o SageMaker Feature Store como um destino no Amazon SageMaker Studio usando a opção **Add destination** (Adicionar destino). Ao concluir as etapas de pré-processamento no SageMaker Data Wrangler, escolha **Add destination** (Adicionar destino) no fluxo. O SageMaker Studio mostra como criar o grupo de recursos e como realizar as etapas necessárias para fazer a ingestão dos dados pré-processados no SageMaker Feature Store.

Consulte [Criar e armazenar recursos com facilidade no Amazon SageMaker sem código](https://aws.amazon.com/blogs/machine-learning/easily-create-and-store-features-in-amazon-sagemaker-without-code/#save_features_to_feature_store) para obter mais informações de como adicionar o SageMaker Feature Store como um destino.

Consulte [Usar o Amazon SageMaker Feature Store com o Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html) para obter mais informações de como criar um grupo de recursos no RSageMaker Studio.

### Exportar um notebook do SageMaker Data Wrangler

Você pode criar grupos de recursos usando a opção **Export to** (Exportar para) no SageMaker Studio. Ao usar a opção **Export to** (Exportar para), você pode criar um notebook que inclua todos os comandos necessários para criar um grupo de recursos.

Com algumas personalizações, você percorrer o notebook para fazer o seguinte:
- Criar uma definição de recurso com base no conjunto de dados.
- Criar um grupo de recursos usando a definição de recurso.
- Armazenar o grupo de recursos no SageMaker Feature Store.
- Configurar as entradas e saídas de um trabalho de processamento.

Este laboratório é semelhante ao notebook exportado. Este laboratório se concentra na primeira parte do notebook exportado. Ele mostra como ingerir dados no grupo de recursos e extrair registros de um armazenamento on-line e de um off-line.

### Usar o SDK Python do SageMaker em um notebook personalizado

Neste laboratório, você vai criar um grupo de recursos e extrair registros de um armazenamento on-line e de um off-line. Você vai aprender como o SageMaker Feature Store funciona com um notebook personalizado. Primeiro, você vai configurar o ambiente na Tarefa 2. Depois, você vai realizar as seguintes tarefas:

Tarefa 2.3: configurar o SageMaker Feature Store
- Criar recursos em um arquivo de notebook.
- Criar um grupo de recursos no SageMaker Feature Store.
- Confirmar se o grupo de recursos foi criado.
- Visualizar o grupo de recursos no SageMaker Studio.

Tarefa 2.4: Consultar os armazenamentos on-line e off-line
- Fazer a ingestão de dados em um grupo de recursos.
- Extrair registros de um armazenamento on-line.
- Extrair registros de um armazenamento off-line usando o Amazon Athena.

## Tarefa 2.2: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies

import boto3
import json
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time
import uuid
import random
from sagemaker.session import Session
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup


region = boto3.Session().region_name
sess = boto3.Session(region_name=region)
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

Importe o conjunto de dados processado de clientes.

In [ ]:
#explore-dataset

column_list = ['income','age','education','education_num','capital_gain','capital_loss','hours_per_week','sex','workclass','marital_status','occupation','relationship','race']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

Depois, examine uma amostra do conjunto de dados em uma tabela.

In [ ]:
#view-dataset

lab_test_data.dtypes
lab_test_data.head()

## Tarefa 2.3: Configurar o SageMaker Feature Store

Crie recursos para ajudar a treinar o modelo. Use os dados processados do laboratório do Amazon SageMaker Data Wrangler para criar um grupo de recursos usando o SageMaker Feature Store.

- Criar recursos em um arquivo de notebook.
- Criar um grupo de recursos no SageMaker Feature Store.
- Confirmar se o grupo de recursos foi criado.
- Visualizar o grupo de recursos no SageMaker Studio.

### Tarefa 2.3.1: Criar recursos em um arquivo de notebook

Para criar um grupo de recursos, você precisa de colunas para atribuir a **record_identifier_name** e **event_time_feature_name**. Para cumprir esse requisito, adicione as colunas **record** e **event_time** ao conjunto de dados.
- **record_identifier_name** se refere a um dos nomes de um recurso definido nas definições de recurso do grupo de recursos. Neste laboratório, você vai criar uma coluna de IDs exclusivos chamada **record**.
- **event_time_feature_name** é um ponto no tempo quando ocorre um novo que corresponde à criação ou à atualização de um registro do grupo de recursos. Todos os registros no grupo de recursos precisam ter um horário do evento correspondente. Ele pode ser usado para monitorar as alterações em um registro ao longo do tempo. Neste laboratório, você vai criar uma coluna chamada **event_time**.

Consulte [Introdução ao Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-concepts.html) para saber mais sobre o nome do identificador de registro ou o horário do evento.

In [ ]:
#add-required-columns

# Add record and event_time columns
current_time_sec = int(round(time.time()))
lab_test_data.insert(0, 'record', range(0, 0 + len(lab_test_data)))
lab_test_data.insert(1, 'event_time', [current_time_sec]*len(lab_test_data))
lab_test_data['record'] = lab_test_data['record'].astype('string')
lab_test_data['event_time'] = lab_test_data['event_time'].astype('float64')

# Set the record-and-event_time-feature-names
record_identifier_feature_name = 'record'
event_time_feature_name = 'event_time'

# View the dataset
print(lab_test_data.dtypes)
lab_test_data.head()

Você pode adicionar novos recursos a qualquer momento durante o processamento de dados, dependendo das necessidades, usando o SDK Python do SageMaker ou o SageMaker Studio.

Neste laboratório, como parte das etapas de processamento, você vai adicionar um recurso usando o SDK Python do SageMaker. Esse recurso é uma combinação ponderada de duas colunas no conjunto de dados e ajuda a treinar o modelo com mais eficiência.

Crie um recurso prático combinando as colunas age e hours_per_week para identificar clientes que estão avançados na carreira.

In [ ]:
#add-feature

lab_test_data = lab_test_data.assign(
    workability = 0.5*lab_test_data.age + 0.5*lab_test_data.hours_per_week)

lab_test_data.head()

### Tarefa 2.3.3: Criar um grupo de recursos no SageMaker Feature Store

Para fazer a ingestão de recursos no SageMaker Feature Store, primeiro exclua as definições de recursos (nome do recurso e tipo de dados) de todos os recursos que pertencem ao grupo de recursos.

Um só recurso corresponde a uma coluna no conjunto de dados. Um grupo de recursos é um esquema predefinido de uma coleção de recursos. Cada recurso no grupo de recursos tem um tipo e um nome de dados especificados. Um único registro em um grupo de recursos corresponde a uma linha no dataframe. Um Feature Store é uma coleção de grupos de recursos. 

Consulte [Criar, armazenar e compartilhar recursos com o Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) para saber mais sobre o SageMaker Feature Store.

Para iniciar o processo de criação de uma definição de recurso, liste o esquema de cada recurso.

In [ ]:
#list-column-schemas

column_schemas = [
    {
        "name": "record",
        "type": "string"
    },
    {
        "name": "event_time",
        "type": "float"
    },
    {
        "name": "income",
        "type": "string"
    },
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "education",
        "type": "float"
    },
    {
        "name": "education_num",
        "type": "float"
    },
    {
        "name": "capital_gain",
        "type": "long"
    },
    {
        "name": "capital_loss",
        "type": "long"
    },
    {
        "name": "hours_per_week",
        "type": "long"
    },
    {
        "name": "sex",
        "type": "float"
    },
    {
        "name": "workclass",
        "type": "array"
    },
    {
        "name": "marital_status",
        "type": "array"
    },
    {
        "name": "occupation",
        "type": "array"
    },
    {
        "name": "relationship",
        "type": "array"
    },
    {
        "name": "race",
        "type": "array"
    },
    {
        "name": "workability",
        "type": "float"
    }
]


Agora que o esquema está definido, crie a entrada das definições de recurso. Configure um mapeamento de tipo para os valores flutuantes e longos do conjunto de dados como FRACTIONAL e INTEGRAL.

Depois, crie a definição de recurso configurando os valores de **feature_name** e **feature_type** em todas as colunas no esquema definido.

In [ ]:
#create-feature-definitions

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

No SageMaker Feature Store, os grupos de recursos podem ser somente on-line, somente off-line ou ambos. Neste laboratório, você usa os armazenamentos on-line e off-line, portanto, **enable_online_store** é definido como **True**.
- O armazenamento on-line é indicado principalmente para dar suporte a previsões em tempo real que precisam de leituras com latência de poucos milissegundos e gravações com alto throughput.
- O armazenamento off-line é indicado principalmente para previsões em lote e treinamento do modelo. O armazenamento off-line aceita apenas acréscimos e pode ser usado para armazenar e acessar dados históricos de recurso.

Quando o Feature Store está definido para armazenamento on-line e off-line, todos os recursos que o armazenamento on-line ingere são replicados no armazenamento off-line. 

Configure o grupo de recursos, especificando opções para as seguintes configurações:
- **feature_group_name**: o nome do grupo de recursos
- **feature_store_offline_s3_uri**: o local do bucket do Amazon Simple Storage Service (Amazon S3) em que o SageMaker Feature Store grava dados no armazenamento off-line de um grupo de recursos.
- **enable_online_store**: controla se um armazenamento on-line está ativado ou não

In [ ]:
#configure-feature-store

# flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = "DataWranglerLab"
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

feature_store_offline_s3_uri = 's3://' + bucket
enable_online_store = True

Agora que o Feature Store está configurado, definida uma Região da AWS e inicie uma sessão. Depois, configure um cliente do SageMaker e um runtime do Feature Store. Por fim, configure a sessão do Feature Store.

Durante a configuração de uma sessão do Feature Store, os valores de **boto_session**, **sagemaker_client** e **sagemaker_featurestore_runtime_client** são definidos.

In [ ]:
#set-up-sagemaker-feature-store-session

sagemaker_client = sess.client(service_name='sagemaker', region_name=region)
featurestore_runtime = sess.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=sess,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

Inicialize o grupo de recursos usando os parâmetros que já foram configurados e invoque a API do Feature Store para criar o grupo de recursos.

In [ ]:
#initialize-feature-group

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=enable_online_store
)

### Tarefa 2.3.3: Confirmar se o grupo de recursos foi criado

O grupo de recursos está pronto agora. Confirme se o grupo de recursos foi criado com sucesso.

Espere até que o grupo de recursos fique pronto usando a API Describe. Essa função verifica a resposta retornada pela API Describe e espera o status **Created** (Criado).

In [ ]:
#wait-for-describe

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

Liste os grupos de recursos usando a API ListFeatureGroups.

In [ ]:
#list-feature-groups

response = sagemaker_client.list_feature_groups()
print(json.dumps(response, indent=4, sort_keys=True, default=str))

### Tarefa 2.3.4: Visualizar o grupo de recursos no SageMaker Studio

Você criou um grupo de recursos do SageMaker usando o SDK Python do SageMaker. Agora, examine o grupo de recursos no SageMaker Studio para descobrir mais detalhes.

1. Na aba de navegação à esquerda do notebook, selecione o ícone **Página inicial do SageMaker**.

A próxima etapa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_5.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_5.ipynb** e selecione **Nova visualização do notebook**. Agora, as orientações ficam visíveis enquanto você explora o grupo de recursos.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar o grupo de recursos, retorne ao notebook selecionando a aba **lab_5.ipynb**.

2. Expanda seção **Data** (Dados) e escolha **Feature Store**.
3. O grupo de recursos recém-criado aparece na aba Feature Store. Você pode examinar os detalhes do grupo de recursos. Para encontrar mais detalhes, escolha o grupo de recursos que começa com **FG-DataWranglerLab-**. Examine os seguintes detalhes enquanto explora o SageMaker Feature Store no SageMaker Studio:
    - **Features** (Recursos): descreve todos os recursos no grupo de recursos, incluindo o **Tipo** e quando o recurso foi criado, com base na coluna **event_time**.
    - **Details** (Detalhes): descreve os metadados do grupo de recursos, incluindo o **Feature group status** (Status do grupo de recursos), o **Record identifier** (Identificador do registro) que você definiu inicialmente no notebook, o **Store type** (Tipo de armazenamento) definido como On-line/Off-line e o **Table name** (Nome da tabela) que você pode usar para consultar dados do Feature Store off-line com o Athena.
    - **Sample query** (Consulta de exemplo): oferece várias consultas de exemplo que você pode usar para consultar dados no Feature Store off-line.

## Tarefa 2.4: Consultar os armazenamentos on-line e off-line

Você criou um grupo de recursos. Agora, você vai fazer a ingestão de dados no grupos recursos e extrair registros de um armazenamento on-line e de um off-line com o Athena.

- Fazer a ingestão de dados em um grupo de recursos.
- Extrair registros de um armazenamento on-line.
- Extrair registros de um armazenamento off-line usando o Athena.

### Tarefa 2.4.1: Ingerir dados em um grupo de recursos

Depois de criar um grupo de recursos, você vai inserir dados nele. Use a chamada da API PutRecord usando **ingest()** com o SDK Python do SageMaker. Sempre que criar um grupo de recursos ou quiser adicionar registros, você vai fazer a ingestão dos recursos no grupo de recursos. 

Para este conjunto de dados, a ingestão exige de três a cinco minutos. A célula é concluída quando uma saída como a seguinte é exibida:

**IngestionManagerPandas(feature_group_name='FG-DataWranglerLab-13ee4f26', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fdb7fccee60>, sagemaker_session=<sagemaker.session.Session object at 0x7fdb82a900d0>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])**

In [ ]:
#ingest-records

feature_group.ingest(data_frame=lab_test_data, wait=True)

### Tarefa 2.4.2: extrair registros de um armazenamento on-line

O armazenamento on-line é útil principalmente porque permite retornar um subconjunto de recursos com rapidez.

Agora que os dados já foram ingeridos, extraia um registro de um armazenamento on-line usando **get_record**.

In [ ]:
#get-record

record = random.randint(0, len(lab_test_data.index)-1)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=str(record))

print(json.dumps(sample_record, indent=4, sort_keys=True, default=str))

Agora, obtenha vários registros do grupo de recursos usando **batch_get_record**. Vários registros já foram escolhidos para você, mas fique à vontade para alterar os registros listados em **records_list**.

In [ ]:
#batch-get-record

records_list = ["7789", "5646", "309", "24528"]

batch_records = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": feature_group_name,
            "RecordIdentifiersValueAsString": records_list,
        }
    ]
)

print(json.dumps(batch_records, indent=4, sort_keys=True, default=str))

### Tarefa 2.4.3 Extrair registros de um armazenamento off-line usando o Athena

Agora que você já extraiu registros de um armazenamento on-line, use o Athena para extrair um armazenamento off-line. 

Você pode consultar o conjunto de dados completo ao treinar e ajustar o modelo ou consultar um subconjunto de registros para inferência. Como o SageMaker Feature Store retém um horário do evento para cada registro, você pode treinar os modelos com o conjunto exato de recursos de um horário anterior específico sem correr o risco de incluir dados além desse horário. 

Como você ajustaria a consulta para alterar o subconjunto de dados retornados do armazenamento off-line?

Primeiro, escolha as configurações da consulta. Você pode personalizar a consulta para examinar qualquer subconjunto dos dados armazenados no grupo de recursos. A consulta a seguir é uma definição básica da consulta SELECT.

In [ ]:
#query-settings
# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup='primary',
        ConfigurationUpdates={
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

#Create the query
query = feature_group.athena_query()
table = query.table_name
query_string = f'SELECT * FROM "{table}" '
output_location = f's3://{bucket}/query_results/'

print(f'Athena query output location: \n{output_location}')

Quando as opções estiverem definidas, execute a consulta e exiba os resultados em formato de tabela.

In [ ]:
#run-athena-query

query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

### Conclusão

Parabéns! Você usou o SageMaker Feature Store para fazer definições de recurso para um grupo de recursos no SageMaker Studio e com o SDK Python do SageMaker. Com o grupo de recursos recém-criado, você já pode treinar e ajustar o modelo usando o SageMaker Experiments no próximo laboratório. Mais adiante no curso, o SageMaker Feature Store também será útil para suplementar dados em solicitações de inferência com a funcionalidade GetRecord de baixa latência usando o armazenamento on-line. Você vai continuar trabalhando com esse conjunto de dados de receita de cliente no próximo laboratório.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.